## View in Colab for latest version:

<a href="https://colab.research.google.com/drive/19mq1tMp_8j7vsq2t2RsrUCI4z2xqaoSQ?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook merges together evictions data with the following tract level data

* CDC health data from the 500 cities dataset
* Food Access Research Atlas data

We also do feature engineering!

In [ ]:
from numpy import loadtxt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read in the Data

In [ ]:
# read in acs data from Google Drive
non_ca = pd.read_csv('/content/drive/MyDrive/MIDS/capstone/data/interim/non_ca_acs_ev.csv',delimiter=",")
ca = pd.read_csv('/content/drive/MyDrive/MIDS/capstone/data/interim/ca_acs.csv',delimiter=",")

In [ ]:
# Read in from local (runtime ~3m6s)  
####################

# ACS
#####
non_ca = pd.read_csv('/content/non_ca_acs_ev.csv')
ca = pd.read_csv('/content/ca_acs.csv')

# CDC
#####
cdc_2016 = pd.read_csv('/content/cdc_health_2016.csv')
cdc_2016["Year"] = [2016 for i in range(len(cdc_2016))]
cdc_2016 = cdc_2016.groupby(['TractFIPS', 'Year']).first() #solves for any duplicated values
cdc_2017 = pd.read_csv('/content/cdc_health_2017.csv')
cdc_2017["Year"] = [2017 for i in range(len(cdc_2017))]
cdc_2017 = cdc_2017.groupby(['TractFIPS', 'Year']).first() #solves for any duplicated values

#this is for 2019 ca
cdc_2019 = pd.read_csv('/content/cdc_health_2019.csv')
cdc_2019["Year"] = [2019 for i in range(len(cdc_2019))]
cdc_2019 = cdc_2019.groupby(['TractFIPS', 'Year']).first() #solves for any duplicated values

# SNAP (Takes a while) 
######
snap_2015 = pd.read_excel('/content/FoodAccessResearchAtlasData2015.xlsx', sheet_name = 'Food Access Research Atlas')
snap_2019 = pd.read_excel('/content/FoodAccessResearchAtlasData2019.xlsx', sheet_name = 'Food Access Research Atlas')


In [ ]:
#create county column
non_ca['County'] = non_ca.GEOID.apply(lambda x: str(x)[0:5]) #find the county id
ca['County'] = ca.GEOID.apply(lambda x: str(x)[0:5]) #find the county id

In [ ]:
#read in cdc data

#this is for 2016 non_ca 
cdc_2016 = pd.read_csv('/content/drive/MyDrive/MIDS/capstone/data/health/cdc_health_2016.csv')
cdc_2016["Year"] = [2016 for i in range(len(cdc_2016))]
cdc_2016 = cdc_2016.groupby(['TractFIPS', 'Year']).first() #solves for any duplicated values
cdc_2017 = pd.read_csv('/content/drive/MyDrive/MIDS/capstone/data/health/cdc_health_2017.csv')
cdc_2017["Year"] = [2017 for i in range(len(cdc_2017))]
cdc_2017 = cdc_2017.groupby(['TractFIPS', 'Year']).first() #solves for any duplicated values

#this is for 2019 ca
cdc_2019 = pd.read_csv('/content/drive/MyDrive/MIDS/capstone/data/health/cdc_health_2019.csv')
cdc_2019["Year"] = [2019 for i in range(len(cdc_2019))]
cdc_2019 = cdc_2019.groupby(['TractFIPS', 'Year']).first() #solves for any duplicated values


In [ ]:
#note this takes a while to read it all in...
snap_2015 = pd.read_excel('/content/drive/MyDrive/MIDS/capstone/data/snap/FoodAccessResearchAtlasData2015.xlsx', sheet_name = 'Food Access Research Atlas')

In [ ]:
snap_2019 = pd.read_excel('/content/drive/MyDrive/MIDS/capstone/data/snap/FoodAccessResearchAtlasData2019.xlsx', sheet_name = 'Food Access Research Atlas')

# CDC Data Join



## Perform Health Data Join

In [ ]:
health = pd.concat([cdc_2016, cdc_2017]).reset_index()

In [ ]:
check = health.groupby(['TractFIPS', 'Year']).count()

if 2 in check.values:
  print("Duplicates exists")

In [ ]:
non_ca = pd.merge(non_ca, health,  how='left', left_on=['GEOID', 'Year'], right_on = ['TractFIPS', 'Year'])
ca =  pd.merge(ca, cdc_2019,  how='left', left_on=['GEOID', 'Year'], right_on = ['TractFIPS', 'Year'])

In [ ]:
non_ca = non_ca[non_ca.columns.drop(list(non_ca.filter(regex='Crude95CI')))] #drop CI 
ca = ca[ca.columns.drop(list(ca.filter(regex='Crude95CI')))] #drop CI

In [ ]:
non_ca.columns.to_list()

['GEOID',
 'Region',
 'Year',
 'Renters',
 'Eviction_Filings',
 'Evictions',
 'Eviction_Filings_RR',
 'Evictions_RR',
 'Tract',
 'Total',
 'Rent',
 'Total_WHITE',
 'Rent_WHITE',
 'Total_BLACK',
 'Rent_BLACK',
 'Total_AI',
 'Rent_AI',
 'Total_ASIAN',
 'Rent_ASIAN',
 'Total_NHPI',
 'Rent_NHPI',
 'Total_OTHER',
 'Rent_OTHER',
 'Total_TWO',
 'Rent_TWO',
 'Total_WHITE_NL',
 'Rent_WHITE_NL',
 'Total_LATINX',
 'Rent_LATINX',
 'MedRent',
 'MHHInc',
 'rb_tot',
 'rb_10.0',
 'rb_14.9',
 'rb_19.9',
 'rb_24.9',
 'rb_29.9',
 'rb_34.9',
 'rb_39.9',
 'rb_49.9',
 'rb_55',
 'rb_nc',
 'totrace',
 'White',
 'Black',
 'Asian',
 'Latinx',
 'totwelf',
 'welf',
 'totpov',
 'povfamh',
 'povnonfamh',
 'totenroll',
 'colenroll',
 'proenroll',
 'HHInc_Total',
 'HHInc_10',
 'HHInc_15',
 'HHInc_20',
 'HHInc_25',
 'HHInc_30',
 'HHInc_35',
 'HHInc_40',
 'HHInc_45',
 'HHInc_50',
 'HHInc_60',
 'HHInc_75',
 'HHInc_100',
 'HHInc_125',
 'HHInc_150',
 'HHInc_200',
 'HHInc_250',
 'HHIncTen_Total',
 'HHIncTenOwn',
 'HHIncTen

## Impute missing health data

In [ ]:
#non-ca df
h = non_ca.iloc[:, 113:-2] #grab only health data
h['Region'] = non_ca.County
avgs = h.groupby('Region').transform(lambda x: x.fillna(x.mean())) #fill na by county means

#merge it all back together
cols_to_use = non_ca.columns.difference(avgs.columns)
non_ca = pd.merge(non_ca[cols_to_use], avgs, left_index=True, right_index=True, how='outer')

In [ ]:
#ca df
h = ca.iloc[:, 100:-2] #grab only health data
h['Region'] = ca.County
avgs = h.groupby('Region').transform(lambda x: x.fillna(x.mean())) #fill na by means

#merge it all back together
cols_to_use = ca.columns.difference(avgs.columns)
ca = pd.merge(ca[cols_to_use], avgs, left_index=True, right_index=True, how='outer')

In [ ]:
#sanity check
diff = non_ca.columns.difference(ca.columns)
diff

Index(['Eviction_Filings', 'Eviction_Filings_RR', 'Evictions', 'Evictions_RR',
       'Region', 'Renters', 'Tract', 'TractFIPS', 'population_count'],
      dtype='object')

# Food Access Research Data Join

In [ ]:
non_ca = non_ca.merge(snap_2015, left_on = "GEOID", right_on= 'CensusTract', how = 'left')
ca = ca.merge(snap_2019.fillna(0), left_on = "GEOID", right_on= 'CensusTract', how = 'left')

# Feature Engineering

## Feature Engineering Function

In [ ]:
def feat_eng(df):

  # add race ratio columns 
  # 4 race categories in data
  # 0/0 produces an NaN, set all NaN to 0 (if zero NHPI, then 0% of NHPI are in tract)

  df['white_ratio']=(df.White/df.totrace).fillna(0)
  df['black_ratio']=(df.Black/df.totrace).fillna(0)
  df['asian_ratio']=(df.Asian/df.totrace).fillna(0)
  df['latinx_ratio']=(df.Latinx/df.totrace).fillna(0)
  df=df.drop(columns=['totrace','White','Black',
                      'Asian','Latinx'])
  

  # add race rent ratio columns
  # 0/0 produces an NaN, set all NaN to 0 (if zero NHPI, then 0% of NHPI are renters)

  df['white_rent_ratio'] = (df.Rent_WHITE/df.Total_WHITE).fillna(0)
  df['black_rent_ratio'] = (df.Rent_BLACK/df.Total_BLACK).fillna(0)
  df['asian_rent_ratio'] = (df.Rent_ASIAN/df.Total_ASIAN).fillna(0)
  df['ai_ratio'] = (df.Rent_AI/df.Total_AI).fillna(0)
  df['nhpi_ratio'] = (df.Rent_NHPI/df.Total_NHPI).fillna(0)


  df=df.drop(columns=['Total_WHITE','Total_BLACK','Total_ASIAN',
                      'Total_AI', 'Total_NHPI', 'Rent_WHITE','Rent_BLACK',
                      'Rent_ASIAN','Rent_AI', 'Rent_NHPI'])
  
  # add household income ratio columns 
  # each of these is a level, divide by rb_total to normalize
  df['rb10']=df["rb_10.0"]/df["rb_tot"]
  df['rb14.9']=df["rb_14.9"]/df["rb_tot"]
  df['rb19.9']=df["rb_19.9"]/df["rb_tot"]
  df['rb24.9']=df["rb_24.9"]/df["rb_tot"]
  df['rb29.9']=df["rb_29.9"]/df["rb_tot"]
  df['rb34.9']=df["rb_34.9"]/df["rb_tot"]
  df['rb39.9']=df["rb_39.9"]/df["rb_tot"]
  df['rb49.9']=df["rb_49.9"]/df["rb_tot"]
  df['rb55']=df["rb_55"]/df["rb_tot"]

  df=df.drop(columns=['rb_10.0','rb_14.9','rb_19.9',
                      'rb_24.9','rb_29.9','rb_34.9',
                      'rb_39.9','rb_49.9','rb_55','rb_tot'])
  
  df['unemployed_rate']=df["unemployed"]/df["labor_force"]
  # Maybe the size of labor force would be important? Have to drop one .
  df = df.drop(columns=['unemployed'])
  return df

In [ ]:
train = feat_eng(non_ca)

In [ ]:
predict = feat_eng(ca)

## Add outcome to training data

In [ ]:
# Remove missing data
train.dropna(subset=['Evictions', 'Renters'])

# create ev_rate column
# Fill with 0 any NaN produced from 0/0
train['ev_rate']= (train.Evictions/train.Renters).fillna(0)

# categorical Outcomes
ev_categories = [1,2,3,4]
train['ev_label'] = pd.cut(train['ev_rate'], bins=(-.01, .01, .03, .05, .13), labels=ev_categories)

In [ ]:
# 3 Categorical Outcomes
train['ev_label'] = pd.cut(train['ev_rate'], bins=(-.01, .02, .05, .13), labels= ['Under 2%', '2%-5%', 'Over 5%'])
train['ev_label'].value_counts(sort=False)

Under 2%    5462
2%-5%       3395
Over 5%     1747
Name: ev_label, dtype: int64

In [ ]:
# Ordinal encoded

# create a list of our conditions
conditions = [
    (train['ev_rate'] < .02),
    (train['ev_rate'] >= .01) & (train['ev_rate'] <= .05),
    (train['ev_rate'] > .05)
    ]

# create a list of the values we want to assign for each condition
values = [1, 2, 3]

train['ord_bin'] = np.select(conditions, values)

train['ord_bin'].value_counts(sort=False)

1    5452
2    3405
3    1994
Name: ord_bin, dtype: int64

In [ ]:
# One hot encoded
train['Under 2%'] = np.where(train['ev_rate'] < .02, 1, 0)
train['2%-5%'] = np.where((train['ev_rate'] >= .02) & (train['ev_rate'] <= .05) , 1, 0)
train['Over 5%'] = np.where((train['ev_rate'] > .05), 1, 0)

print('Under 2%', sum(train['Under 2%']), '\n',
      '2%-5%', sum(train['2%-5%']), '\n',
      'Over 5%', sum(train['Over 5%'])
      )

Under 2% 5452 
 2%-5% 3405 
 Over 5% 1994


In [ ]:
train = train.drop(columns=['POP2010', 'Renters','population_count',
                      'Tract',
                      'StateAbbr',
                      'PlaceName',
                      'PlaceFIPS',
                      'Place_TractID',
                      'Eviction_Filings', #Related to outcome
                      'Evictions',
                      'Eviction_Filings_RR',
                      'Evictions_RR',
                      'Region', # From Health join
                      'Geolocation',
                      'TractFIPS',
                      'Population2010',
                      'CensusTract', # From SNAP join
                      'State',
                      'County_y', 'County_x'])

In [ ]:
predict = predict.drop(columns=[
                      'CensusTract', 'County_x', 'County_y', 'Geolocation', 'PlaceFIPS',
       'PlaceName', 'Place_TractID', 'Pop2010', 'Population2010', 'State',
       'StateAbbr', 'tract'])

In [ ]:
#sanity check
diff = train.columns.difference(predict.columns)
diff

Index(['2%-5%', 'Over 5%', 'Under 2%', 'ev_label', 'ev_rate', 'ord_bin'], dtype='object')

In [ ]:
#sanity check
diff = predict.columns.difference(train.columns)
diff

Index([], dtype='object')

# Output to csvs

In [ ]:
train.to_csv('non_ca.csv')
predict.to_csv('ca.csv')

!cp non_ca.csv "/content/drive/MyDrive/MIDS/capstone/data/processed"
!cp ca.csv "/content/drive/MyDrive/MIDS/capstone/data/processed"

cp: cannot create regular file '/content/drive/MyDrive/MIDS/capstone/data/processed': No such file or directory
cp: cannot create regular file '/content/drive/MyDrive/MIDS/capstone/data/processed': No such file or directory


In [ ]:
#
train.to_csv('non_ca.csv')
predict.to_csv('ca.csv')

In [ ]:
ca.columns

Index(['Asian', 'Black', 'GEOID', 'Geolocation', 'HHIncTenOwn',
       'HHIncTenOwn_10', 'HHIncTenOwn_100', 'HHIncTenOwn_15',
       'HHIncTenOwn_150', 'HHIncTenOwn_151',
       ...
       'TractNHOPI', 'TractAIAN', 'TractOMultir', 'TractHispanic', 'TractHUNV',
       'TractSNAP', 'white_ratio', 'black_ratio', 'asian_ratio',
       'latinx_ratio'],
      dtype='object', length=287)

In [ ]:
non_ca.columns

Index(['Asian', 'Black', 'County_x', 'Eviction_Filings', 'Eviction_Filings_RR',
       'Evictions', 'Evictions_RR', 'GEOID', 'Geolocation', 'HHIncTenOwn',
       ...
       'TractNHOPI', 'TractAIAN', 'TractOMultir', 'TractHispanic', 'TractHUNV',
       'TractSNAP', 'white_ratio', 'black_ratio', 'asian_ratio',
       'latinx_ratio'],
      dtype='object', length=295)